In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [5]:
## get intermediate layers of pytorch model

class LayerNet(Net):

    def __init__(self):
        super().__init__()

    def forward(self, x):
        x1 = F.relu(F.max_pool2d(self.conv1(x), 2))
        x2 = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x1)), 2))
        x3 = x2.view(-1, 320)
        x4 = F.relu(self.fc1(x3))
        x5 = F.dropout(x4, training=self.training)
        x6 = self.fc2(x5)
        return [x1, x2, x3, x4, x5, x6, F.log_softmax(x6, dim=1)]

In [6]:
import argparse
import torch.optim as optim
from torchvision import datasets, transforms

def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)[-1]
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)[-1]
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# Training settings
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')

# Train this model with 60 epochs and after process every 300 batches log the train status
args = parser.parse_args(['--epochs', '10', '--log-interval', '300'])

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


model = LayerNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)



In [7]:
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.371851
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.748811
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.733573
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.476532

Test set: Average loss: 0.2095, Accuracy: 9402/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.357870
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.460683
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.324104
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.294746

Test set: Average loss: 0.1262, Accuracy: 9618/10000 (96%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.447501
Train Epoch: 3 [19200/60000 (32%)]	Loss: 0.429467
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.482324
Train Epoch: 3 [57600/60000 (96%)]	Loss: 0.379503

Test set: Average loss: 0.1017, Accuracy: 9692/10000 (97%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.205312
Train Epoch: 4 [19200/60000 (32%)]	Loss: 0.278617
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.328676
Train Epoch: 4 [57600/60000 (96%)]	Loss: 0.195698

Test set: Average loss: 0.0851, Accu

In [8]:
# Save the trained model to a file
torch.save(model.state_dict(), 'models/mnist_pytorch_24_06_22.pth')

In [9]:
from Main import translate
tf_rep = translate(model, 'tf2', dummy_input=torch.randn([64, 1, 28, 28]))

INFO:tensorflow:Assets written to: models/mnist_tf.pb\assets


INFO:tensorflow:Assets written to: models/mnist_tf.pb\assets


In [18]:
import tensorflow as tf
print(tf_rep.signatures)
print(tf_rep.signatures['serving_default'])
tf_layers = tf.keras.Model(inputs=tf_rep.inputs, outputs=[tf_rep.layers[1].output]) + tf_rep.outputs

_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(*, input.1) at 0x1D7C866E100>})
ConcreteFunction signature_wrapper(*, input.1)
  Args:
    input.1: float32 Tensor, shape=(64, 1, 28, 28)
  Returns:
    {'20': <1>, 'input': <2>, 'onnx::Gemm_16': <3>, 'onnx::Reshape_14': <4>, 'x4': <5>, 'x6': <6>}
      <1>: float32 Tensor, shape=(64, 10)
      <2>: float32 Tensor, shape=(64, 10, 12, 12)
      <3>: float32 Tensor, shape=(64, 320)
      <4>: float32 Tensor, shape=(64, 20, 4, 4)
      <5>: float32 Tensor, shape=(64, 50)
      <6>: float32 Tensor, shape=(64, 10)


AttributeError: '_UserObject' object has no attribute 'inputs'

In [25]:
import matplotlib.pyplot as plt
import numpy as np

features, labels = next(iter(train_loader))
tf_logits_full = tf_rep(**{'input.1':features})
tf_logits = tf_logits_full['20']
tf_input_layer = tf_logits_full['input']
tf_output = np.argmax(tf_logits, axis=1)
torch_logits_full = model(features)
torch_logits = torch_logits_full[-1]
torch_input_layer = torch_logits_full[0]
torch_output = torch.argmax(torch_logits, dim=1)
for torch_l, torch_o, tf_l, tf_o, y, torch_i, tf_i in zip(torch_logits, torch_output, tf_logits, tf_output, labels.tolist(), torch_input_layer,  tf_input_layer):
        print(f'torch-output: {torch_o}, tf-output: {tf_o}, label: {y}')
        print(f'torch-logits: {torch_l}')
        print(f'tf-logits: {tf_l}')
        print(f'torch-input-layer: {torch_i[0]}')
        print(f'tf-input-layer: {tf_i[0]}')

torch-output: 3, tf-output: 3, label: 7
torch-logits: tensor([-4.8913, -5.8551, -6.8544, -1.0581, -4.7200, -1.3944, -6.6220, -1.3009,
        -5.5388, -2.2348], grad_fn=<UnbindBackward0>)
tf-logits: [-4.891301  -5.8551292 -6.8544235 -1.0580844 -4.7200184 -1.3943976
 -6.6220417 -1.3009256 -5.5387645 -2.2348418]
torch-input-layer: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000],
        [0.0000, 0.8272, 2.2585, 3.5684, 2.7012, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000],
        [0.2670, 1.1438, 2.8266, 4.7175, 5.0853, 3.9498, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000],
        [0.6148, 0.2850, 0.0000, 0.2655, 3.9571, 5.6339, 5.4740, 0.4251, 0.0000,
         0.0000, 0.0000, 0.0000],
        [0.4290, 0.0000, 0.6234, 0.0000, 0.0000, 2.5210, 5.2139, 5.7413, 2.6226,
         0.00